# HuggingFace BERT model inference with TensorFlow

This notebook is the inference of [BERT fine-tuning notebook](https://www.kaggle.com/code/electro/fp3-bert-fine-tuning-tensorflow).

I also have [a basic EDA and bag-of-words solution](https://www.kaggle.com/code/electro/fp3-bag-of-words-tensorflow-starter). Please check it out if you are interested.

If you find this notebook helpful, please upvote it. Thank you.

# Imports

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

# Config

In [2]:
TARGET_COLS = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

MAX_LENGTH = 512
BATCH_SIZE = 1
OFFLINE_TOKENIZER = "../input/tfbert-base-uncased"

# Load Data

In [3]:
test_df = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
test_df.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [4]:
#https://keras.io/examples/nlp/semantic_similarity_with_bert/
class BertDataGenerator(tf.keras.utils.Sequence):
    def __init__(
        self,
        full_texts,
        labels=None,
        batch_size=BATCH_SIZE,
        shuffle=False,
        include_targets=False,
    ):
        self.full_texts = full_texts
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            OFFLINE_TOKENIZER, do_lower_case=True
        )
        self.indexes = np.arange(len(self.full_texts))
        self.on_epoch_end()

    def __len__(self):
        return len(self.full_texts) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_texts = self.full_texts[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            batch_texts.tolist(),
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="tf",
            truncation=True,
            padding='max_length'
        )

        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="float32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [5]:
test_data = BertDataGenerator(
    test_df["full_text"].values.astype("str"),
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Load Model

In [6]:
model = tf.keras.models.load_model('../input/fp3-model-bert-public/bert-finetuning', compile=False)

2022-10-09 10:05:58.029108: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 10:05:58.147897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 10:05:58.148671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-09 10:05:58.150609: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
preds = model.predict(test_data)
preds = np.clip(preds, 1, 5)

2022-10-09 10:06:18.746266: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


# Submission

In [8]:
sub_df = pd.concat([test_df[['text_id']], pd.DataFrame(preds, columns=TARGET_COLS)], axis=1)
sub_df.to_csv('submission.csv', index=False)

In [9]:
sub_df.head()

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.929134,2.913305,3.260777,3.229188,2.817615,2.923548
1,000BAD50D026,2.627238,2.462691,2.695321,2.423433,2.231280,2.732735
2,00367BB2546B,3.711233,3.604190,3.593739,3.573730,3.421958,3.872856
